## Naver API 포스터, 평점 정보 로드

In [1]:
import os
import sys
import time
import json

import urllib
import urllib.request
import requests

import pandas as pd
from bs4 import BeautifulSoup

In [2]:
data_path = r"C:\Users\Jwp\Desktop\workspace\Final_project\DataScience\Final_Data\movie\movieInfo\all"
save_path = r"C:\Users\Jwp\Desktop\workspace\Final_Project\DataScience\Final_Data\movie\moviePoster"

In [18]:
movies = pd.read_csv(os.path.join(data_path, "movie_merged_all_en.csv"), encoding="utf-8")

In [19]:
len(movies)

19874

In [20]:
movies.tail()

,movieCd,movieNm,movieNmEn,prdtYear,openDt,prdtStatNm,repGenre,genre,repNation,nations,...,directors,actors,actors_en,cast,cast_en,companyCd,companyNm,companyNmEn,companyPartNm,watchGradeNm
19869,19910228,LA이야기,La Story,1991,19910608,개봉,코미디,코미디,미국,미국,...,믹 잭슨,"['스티브 마틴', '빅토리아 테넌트']","['Steve Martin', 'Victoria Tennant']","['', '']","['', '']",['20100963'],['세경영화㈜'],[''],['수입사'],['고등학생이상관람가']
19870,19920137,우리는 그들을 잊으려 했다,THE FORGOTTEN,1992,19920307,개봉,NaN,NaN,NaN,NaN,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19871,19880146,원+씩스,ONE + SIX,1988,19880625,개봉,NaN,NaN,이탈리아,이탈리아,...,,[],[],[],[],[],[],[],[],[]
19872,19910263,최가손우2,CRAZY COMPANIES 2,1988,19910803,개봉,NaN,NaN,NaN,NaN,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19873,19910096,칙칙이의 내일은 참피온,Tomorrow's Champion,1991,19910803,개봉,코미디,코미디,한국,한국,...,,['장고웅'],['JANG Go-ung '],[''],[''],[],[],[],[],[]


In [6]:
def progress_bar(i, total):
    k = i/total*100
    star = "*"*int(k/2.5)
    print("Processing: {}/{} [{:.1f}%] {}".format(i, total, k, star), end="\r")

In [7]:
def get_naverApi(movieNm, openDt, director):
    
    client_id = "JjcGyzzf5rjyFKNPCTxG"
    client_secret = "MDi_1o76AT"
    encText = urllib.parse.quote(f"{movieNm}")
    url = "https://openapi.naver.com/v1/search/movie?query=" + encText + f"&yearfrom={openDt-1}&yearto={openDt}"
    request = urllib.request.Request(url)

    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)

    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if(rescode == 200):
        html = response.read().decode("utf-8")
        parse = json.loads(html)
                
        if parse["items"] == []:
            poster = ""
            userRating = ""
            
        else:
            if len(parse["items"]) > 1:
                for i, item in enumerate(parse["items"]):
                    if director in item["director"]:
                        idx = i
                        break
                    elif item["pubDate"] == str(openDt):
                        idx = i
                        break
                    else:
                        idx = 0
            else:
                 idx = 0
            
            info = parse["items"][idx]
            poster = info["image"]
            userRating = info["userRating"]
            
    else:
        poster = ""
        userRating = ""
    
    return poster, userRating

In [206]:
image, score = get_naverApi("졸업", 2019, "박주환")

In [207]:
image, score

('https://ssl.pstatic.net/imgmovie/mdi/mit110/1737/173753_P10_104955.jpg',
 '10.00')

In [10]:
def load_naverMovie(movie_list):
    movie_list.openDt = movie_list.openDt.astype(str)
    ls = movie_list[["movieNm", "openDt", "directors"]].values
    
    i = 1
    total = len(ls)
    poster = []
    userRating = []
    for v in ls:
        image, score = get_naverApi(v[0], int(v[1][:4]), v[2].split(" ")[0].split(",")[0][:2])
        poster.append(image)
        userRating.append(score)
        
        progress_bar(i, total)
        i += 1
        time.sleep(0.1)

    print()
    print("complete!")
    
    return poster, userRating

### 포스터, 평점 정보 불러오기

In [11]:
poster, userRating = load_naverMovie(movies)

Processing: 19874/19874 [100.0%] ****************************************
complete!


In [12]:
naver_movie = pd.DataFrame([poster, userRating], index=["poster", "userRating"]).T

In [13]:
len(naver_movie)

19874

In [14]:
naver_movie[naver_movie.poster == ""]

,poster,userRating
11,,0.00
12,,0.00
22,,
24,,
28,,
...,...,...
19868,,7.50
19870,,
19871,,
19872,,


In [15]:
naver_movie.to_csv(os.path.join(save_path, "naver_movie2.csv"), index=None, encoding="utf-8")

In [216]:
# naver_movie.to_csv(os.path.join(save_path, "naver_movie_euc.csv"), index=None, encoding="euc-kr")

## Error 데이터 확인

In [189]:
movieNm = "크롤"
openDt = 2019

In [190]:
client_id = "JjcGyzzf5rjyFKNPCTxG"
client_secret = "MDi_1o76AT"
encText = urllib.parse.quote(f"{movieNm}")
url = "https://openapi.naver.com/v1/search/movie?query=" + encText + f"&yearfrom={openDt-1}&yearto={openDt}"
request = urllib.request.Request(url)

request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)

response = urllib.request.urlopen(request)
rescode = response.getcode()

if(rescode == 200):
    html = response.read().decode("utf-8")
    parse = json.loads(html)

    if parse["items"] == []:
        poster = ""
        userRating = ""

    else:
        if len(parse["items"]) > 1:
            for i, item in enumerate(parse["items"]):
                if director in item["director"]:
                    idx = i
                elif item["pubDate"] == str(openDt):
                    idx = i
                else:
                    idx = 0
        else:
             idx = 0

        info = parse["items"][idx]
        poster = info["image"]
        userRating = info["userRating"]

else:
    poster = ""
    userRating = ""

In [191]:
parse

{'lastBuildDate': 'Tue, 29 Oct 2019 16:59:10 +0900',
 'total': 3,
 'start': 1,
 'display': 3,
 'items': [{'title': '<b>크롤</b>',
   'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=183803',
   'image': 'https://ssl.pstatic.net/imgmovie/mdi/mit110/1838/183803_P01_165442.jpg',
   'subtitle': 'Crawl',
   'pubDate': '2019',
   'director': '알렉산드르 아야|',
   'actor': '카야 스코델라리오|배리 페퍼|',
   'userRating': '6.52'},
  {'title': '포르<b>크롤</b>',
   'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=186770',
   'image': 'https://ssl.pstatic.net/imgmovie/mdi/mit110/1867/186770_P01_163536.JPG',
   'subtitle': 'Porquerolles',
   'pubDate': '2019',
   'director': '니키 해믈린|',
   'actor': '',
   'userRating': '0.00'},
  {'title': '나이트 <b>크롤</b>러스',
   'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=176474',
   'image': '',
   'subtitle': 'NIGHT CRAWLERS',
   'pubDate': '2018',
   'director': '덴타루스 존스|',
   'actor': '',
   'userRating': '0.00'}]}

In [71]:
parse["items"] == []

True